In [53]:
# coding: utf-8

import pandas as pd
import numpy as np
from matplotlib import rcParams
import matplotlib.pyplot as plt
from collections import defaultdict
from datetime import datetime
import matplotlib.patches as mpatches
import matplotlib
import time
from __future__ import print_function
%matplotlib inline  

rcParams['font.family'] = 'NanumGothic'
rcParams.update({'font.size': 12})
matplotlib.style.use('ggplot')
pd.options.display.max_rows=14

In [2]:
def movieLensDataLoad():
    ## user 영화 별점 data 
    encoding='latin1'
    unames = ['user_id','gender','age','occupation','zip']
    rnames = ['user_id','movie_id','rating','timestamp']
    mnames = ['movie_id','title','genres']
    
    ratings = pd.read_csv("movielens/ratings.dat",sep='::',header=None,names=rnames,encoding=encoding)
    movies = pd.read_csv("movielens/movies.dat",sep='::',header=None,names=mnames,encoding=encoding)
    users = pd.read_csv("movielens/users.dat",sep='::',header=None,names=unames,encoding=encoding)
    return ( ratings, movies, users )

In [3]:
ratings,movies,users = movieLensDataLoad()

C:\Users\Xnote\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
C:\Users\Xnote\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':
C:\Users\Xnote\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  # Remove the CWD from sys.path while we load

In [13]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [7]:
UM_matrix_ds = ratings.pivot(index='user_id', columns='movie_id', values='rating')

print( "UM Matrix value size", UM_matrix_ds.values.size)
print( "ratings value size", ratings.values.size)

UM Matrix value size 22384240
ratings value size 4000836


In [14]:
UM_matrix_ds.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
## 유사도(simularity)
import math
from operator import itemgetter
from scipy.spatial import distance

def distance_cosine(a,b):
    return 1-distance.cosine(a,b)

def disance_corr(a,b):
    return 1-distance.correlation (a,b)

def distance_euclidean(a,b):
    return 1/(distance.euclidean(a,b)+1)

In [51]:
def nearest_neighbor_user( user, topN, simFunc ) :  
    u1 = UM_matrix_ds.loc[user].dropna()
    ratedIndex = u1.index
    nn = {}
    
    ## Brote Force Compute
    for uid, row in UM_matrix_ds.iterrows():
        interSectionU1 = []
        interSectionU2 = []
        if uid==user:
            continue
            
        for i in ratedIndex:
            if False==math.isnan(row[i]):
                interSectionU1.append(u1[i])
                interSectionU2.append(row[i])
        interSectionLen = len(interSectionU1)
        
       
        ## At least 3 intersection items
        if interSectionLen < 3 :
            continue
            
        ## similarity functon
        sim = simFunc(interSectionU1,interSectionU2)
        
        if  math.isnan(sim) == False:
            nn[uid] = sim
            
    ## top N returned         
    return sorted(nn.items(),key=itemgetter(1),reverse=True)[:(topN+1)]
    #return sorted(nn.items(),key=itemgetter(1))[:-(topN+1):-1]

In [54]:
st=time.time()
print(nearest_neighbor_user(8, 50, distance_euclidean))
print(time.time()-st, 'sec')

[(5640, 1.0), (5609, 1.0), (5597, 1.0), (4874, 1.0), (4741, 1.0), (4727, 1.0), (4544, 1.0), (4394, 1.0), (4332, 1.0), (4324, 1.0), (4321, 1.0), (4027, 1.0), (3723, 1.0), (3541, 1.0), (3455, 1.0), (3421, 1.0), (3218, 1.0), (2930, 1.0), (2834, 1.0), (2729, 1.0), (2710, 1.0), (2527, 1.0), (2193, 1.0), (1964, 1.0), (1704, 1.0), (761, 1.0), (206, 1.0), (128, 1.0), (115, 1.0), (71, 1.0), (66, 1.0), (6032, 0.5), (5895, 0.5), (5690, 0.5), (5589, 0.5), (5287, 0.5), (5274, 0.5), (5244, 0.5), (5212, 0.5), (5202, 0.5), (5192, 0.5), (5168, 0.5), (5141, 0.5), (5135, 0.5), (5066, 0.5), (5027, 0.5), (4955, 0.5), (4797, 0.5), (4777, 0.5), (4677, 0.5)]
22.26279926300049 sec


In [78]:
def predictRating(userid, nn=50, simFunc=distance_euclidean) :
   
    ## neighboorhood 
    neighbor = nearest_neighbor_user(userid,nn,simFunc)
    # userid : similarity 의 dictionary
    
    neighbor_id = [id for id,sim in neighbor]
    # 비슷한 유사도를 보이는 유저 리스트
    
    
    ## 4개이상이 NaN인 경우 제거
    neighbor_movie = UM_matrix_ds.loc[neighbor_id].dropna(1, how='all', thresh = 4 )
    # 유저id x 영화id 의 DataFrame. 단, column의 NaN이 4개 이상인 경우 삭제했음.
    
    neighbor_dic = (dict(neighbor))
    ret = [] # ['movieId', 'predictedRate']
    
    # 각 column을 순회한다. key : userid, column : movieid, value : rating
    for movieId, row in neighbor_movie.iteritems():
        jsum, wsum = 0, 0
        for v in row.dropna().iteritems():
            sim = neighbor_dic.get(v[0],0)
            jsum += sim
            wsum += (v[1]*sim)
        ret.append([movieId, wsum/jsum])
       
    return ret

In [79]:
(predictRating(10, 50))

[[34, 3.9684588267299019],
 [110, 4.3380548409552331],
 [111, 3.8660254037844388],
 [260, 4.8004726657018635],
 [318, 4.7818193362997521],
 [480, 4.0],
 [527, 4.3990640578222866],
 [1097, 5.0],
 [1127, 4.2345598077387843],
 [1193, 4.164981414379116],
 [1210, 4.0367101079462504],
 [1617, 3.7868370248847207],
 [1625, 3.0],
 [1721, 3.3854882858175941],
 [1911, 3.6561586730733868],
 [1961, 4.807224924849967],
 [2028, 4.3152873672667029],
 [2336, 4.4885804623238483],
 [2355, 4.5],
 [2394, 4.2345598077387852],
 [2396, 4.6037050236448938],
 [2541, 4.0438227223805256],
 [2605, 4.0446129650861185],
 [2628, 3.4632868920359523],
 [2683, 3.394929500252196],
 [2686, 4.2616864452805139],
 [2688, 3.3380043854351396],
 [2694, 3.5033570020455183],
 [2699, 3.4160774241793734],
 [2706, 4.0],
 [2710, 3.6477806361790717],
 [2712, 3.4459029062228055],
 [2716, 3.5734153602688892],
 [2722, 3.4840678157632725],
 [2724, 3.2525765710743832],
 [2762, 4.82313920350642],
 [2763, 4.0],
 [2770, 4.2574888379355329],
 

In [83]:
## user의 별점 매긴 영화와 영화 정보 높은 별점순으로 보기 
def ratingMovies(user_id):
    ds = pd.merge(ratings[ratings.user_id==user_id], movies, on=['movie_id'])
    return ds.sort(['rating'],ascending=False)[['rating','title','genres','movie_id']].head(100)
ratingMovies(1).head(20)

C:\Users\Xnote\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  after removing the cwd from sys.path.


,rating,title,genres,movie_id
0,5,One Flew Over the Cuckoo's Nest (1975),Drama,1193
46,5,Dumbo (1941),Animation|Children's|Musical,1029
40,5,Toy Story (1995),Animation|Children's|Comedy,1
18,5,Awakenings (1990),Drama,3105
41,5,Rain Man (1988),Drama,1961
23,5,Schindler's List (1993),Drama|War,527
37,5,Cinderella (1950),Animation|Children's|Musical,1022
...,...,...,...,...
48,5,Saving Private Ryan (1998),Action|Drama|War,2028
7,5,"Christmas Story, A (1983)",Comedy|Drama,2804


In [87]:
def join_movie_info( predicted_result ):
    predicted_ratings = pd.DataFrame(predicted_result, columns=['movie_id', 'predicted_rating'])
    result_ds = pd.merge( movies[movies.movie_id > 0], predicted_ratings, on=['movie_id'])
    return result_ds.sort(['predicted_rating'], ascending=False)

result = predictRating(1);
join_movie_info(result)

C:\Users\Xnote\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  after removing the cwd from sys.path.


,movie_id,title,genres,predicted_rating
0,1,Toy Story (1995),Animation|Children's|Comedy,5.000000
41,913,"Maltese Falcon, The (1941)",Film-Noir|Mystery,5.000000
108,2336,Elizabeth (1998),Drama,5.000000
97,1961,Rain Man (1988),Drama,5.000000
28,527,Schindler's List (1993),Drama|War,4.965517
52,1193,One Flew Over the Cuckoo's Nest (1975),Drama,4.944444
102,2028,Saving Private Ryan (1998),Action|Drama|War,4.928571
...,...,...,...,...
103,2108,L.A. Story (1991),Comedy|Romance,2.857143
159,3471,Close Encounters of the Third Kind (1977),Drama|Sci-Fi,2.833333


In [89]:
## 5번 유저의 별점 예측 
userid=1
pd.merge(ratingMovies(userid), join_movie_info(predictRating(userid)), on=['movie_id'], how='right')\
    .sort(['predicted_rating'], ascending =False)\

C:\Users\Xnote\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  after removing the cwd from sys.path.
C:\Users\Xnote\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  This is separate from the ipykernel package so we can avoid doing imports until


,rating,title_x,genres_x,movie_id,title_y,genres_y,predicted_rating
15,NaN,NaN,NaN,913,"Maltese Falcon, The (1941)",Film-Noir|Mystery,5.000000
1,5.0,Toy Story (1995),Animation|Children's|Comedy,1,Toy Story (1995),Animation|Children's|Comedy,5.000000
2,5.0,Rain Man (1988),Drama,1961,Rain Man (1988),Drama,5.000000
16,NaN,NaN,NaN,2336,Elizabeth (1998),Drama,5.000000
3,5.0,Schindler's List (1993),Drama|War,527,Schindler's List (1993),Drama|War,4.965517
0,5.0,One Flew Over the Cuckoo's Nest (1975),Drama,1193,One Flew Over the Cuckoo's Nest (1975),Drama,4.944444
5,5.0,Saving Private Ryan (1998),Action|Drama|War,2028,Saving Private Ryan (1998),Action|Drama|War,4.928571
...,...,...,...,...,...,...,...
161,NaN,NaN,NaN,2108,L.A. Story (1991),Comedy|Romance,2.857143
162,NaN,NaN,NaN,3471,Close Encounters of the Third Kind (1977),Drama|Sci-Fi,2.833333


In [91]:
data2 = pd.DataFrame(np.array([[1,2,3],[4,5,6],[7,8,9]]),columns=['a','b','c'])

In [92]:
data2

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [105]:
data2.loc[(data2.a == 4) & (data2.b == 5),'c']
eval_ratings = ratings

In [110]:
# ratings['userId'].drop_duplicates().values[:]
def eval_prediction( predict_users,  n_users=50 ):
    ## evaluation
    ds = pd.merge(eval_ratings, 
                       ratings[['movie_id','rating']].groupby(['movie_id']).mean().reset_index(), 
                       on='movie_id', how='left')

    ds = ds.rename(columns= {'rating_x':'rating', 'rating_y':'mean_rating'})

    st = time.time()
    ## udpate to predict_rating 
    distance_functions = [ ('euclidean',distance_euclidean), ('cosine', distance_cosine) ]
    for name, func in distance_functions:
        ds[name] = 0
        for user_id in predict_users:
            # key: movie id, value : predicted_rating
            for x in predictRating(user_id, n_users, func):
                ds.loc[(ds.user_id==user_id) & (ds.movie_id==x[0]),name]=x[1]
    print('elapsed', round(time.time()-st,2), 'sec')
    #전체 dataFrame에서 predict를 수행한 user에 해당하는 tuple만 리턴.
    return ds[ds.euclidean+ds.cosine>0]

In [116]:
## 전체 userId list 
users = UM_matrix_ds.index.tolist()
## 10명 별점 예측
predicted = eval_prediction(users[:2], 100 )
predicted

elapsed 100.93 sec


,user_id,movie_id,rating,timestamp,mean_rating,euclidean,cosine
0,1,1193,5,978300760,4.390725,4.838710,4.954572
3,1,3408,4,978300275,3.863878,4.105263,4.199828
4,1,2355,5,978824291,3.854375,4.777778,4.666306
5,1,1197,3,978302268,4.303710,3.181818,3.428669
7,1,2804,5,978300719,4.238905,4.888889,4.750065
9,1,919,4,978301368,4.247963,0.000000,3.747070
13,1,2918,4,978302124,4.117447,0.000000,4.332933
...,...,...,...,...,...,...,...
137,2,2396,4,978299641,4.127480,4.149096,4.317753
141,2,3809,3,978299712,3.397797,3.000000,0.000000


In [117]:
predicted = predicted[ (predicted['cosine'] > 0) & (predicted['euclidean'] > 0) ]
def RMSE(X, left_col, right_col):
    return(np.sqrt(np.mean( (X[left_col] - X[right_col])**2 )))

def MAE(X, left_col, right_col):
    return(np.mean(np.absolute(X[left_col] - X[right_col])) )
for name in ['mean_rating', 'cosine', 'euclidean']:
    print ("MAE of {0} is {1} ".format(name, MAE( predicted, 'rating', name )))

for name in ['mean_rating', 'cosine', 'euclidean']:
    print ("RMSE of {0} is {1} ".format(name, RMSE( predicted, 'rating', name )))

MAE of mean_rating is 0.546920929099824 
MAE of cosine is 0.26650271424539923 
MAE of euclidean is 0.14174123468743638 
RMSE of mean_rating is 0.6430550871260872 
RMSE of cosine is 0.33226879399066883 
RMSE of euclidean is 0.19638788159077566 
